In [ ]:
import pandas as pd
from datetime import datetime

# Definindo o horário comercial
def working_hours(start, end):
    start_time = start.replace(hour=8, minute=0)
    end_time = end.replace(hour=18, minute=0)
    
    # Se a data de início for depois das 18:00, não conta nada
    if start > end_time:
        return 0
    # Se a data de início for antes das 08:00, ajusta
    if start < start_time:
        start = start_time
    # Se a data de fim for antes das 08:00, não conta nada
    if end < start_time:
        return 0
    # Se a data de fim for depois das 18:00, ajusta
    if end > end_time:
        end = end_time
    
    return (end - start).total_seconds() / 3600  # Retorna em horas

# Função para calcular o tempo em horário comercial
def calculate_working_time(row):
    abertura = pd.to_datetime(row['Data de abertura'], format='%d/%m/%Y %H:%M')
    if pd.isna(row['Data da solução']):
        solucoes = datetime.now()  # Se a data da solução estiver vazia, usa a data e hora atual
    else:
        solucoes = pd.to_datetime(row['Data da solução'], format='%d/%m/%Y %H:%M')
    
    total_hours = 0

    # Itera sobre os dias entre as duas datas
    while abertura.date() < solucoes.date():
        next_day = abertura + pd.Timedelta(days=1)
        total_hours += working_hours(abertura, next_day)
        abertura = next_day.replace(hour=8, minute=0)  # Inicia no próximo dia às 08:00

    total_hours += working_hours(abertura, solucoes)  # Adiciona as horas do último dia
    return total_hours

# Dados fornecidos
data = {
    'Data de abertura': [
        '01/10/2024 09:27', '01/10/2024 09:36', '01/10/2024 09:49', 
        '01/10/2024 17:06', '02/10/2024 09:30', '02/10/2024 15:04', 
        '04/10/2024 11:02', '07/10/2024 09:43', '07/10/2024 17:48', 
        '08/10/2024 09:57', '08/10/2024 17:50', '09/10/2024 14:47', 
        '10/10/2024 10:31', '10/10/2024 10:43', '10/10/2024 11:03', 
        '10/10/2024 11:17', '10/10/2024 11:25', '10/10/2024 11:33', 
        '10/10/2024 11:40', '10/10/2024 11:41', '10/10/2024 11:44', 
        '11/10/2024 12:02', '14/10/2024 17:46', '15/10/2024 09:35'
    ],
    'Data da solução': [
        '02/10/2024 16:28', '01/10/2024 16:31', '01/10/2024 16:39', 
        '10/10/2024 14:45', '07/10/2024 13:34', '03/10/2024 11:44', 
        '07/10/2024 14:32', '08/10/2024 14:15', '08/10/2024 11:52', 
        '15/10/2024 14:15', '10/10/2024 09:58', None, 
        '10/10/2024 15:39', '10/10/2024 17:38', '10/10/2024 15:45', 
        '11/10/2024 07:12', '10/10/2024 15:48', '10/10/2024 17:07', 
        '10/10/2024 15:46', '10/10/2024 15:46', '14/10/2024 14:14', 
        None, '15/10/2024 12:59', '15/10/2024 17:32'
    ]
}

# Criando DataFrame
df = pd.DataFrame(data)

# Calculando o tempo em horas de atendimento
df['Tempo em atendimento'] = df.apply(calculate_working_time, axis=1)

df[['Data de abertura', 'Data da solução', 'Tempo em atendimento']]

# Função para estilizar o cabeçalho (preto com texto preto)
def highlight_header(data):
    return ['background-color: black; color: black; font-weight: bold;' for _ in data]

# Função para aplicar cores alternadas nas linhas (zebra)
def zebra_stripes(index):
    if index % 2 == 0:
        return ['background-color: #f0f0f0;'] * len(df.columns)  # Linhas pares - cinza claro
    else:
        return ['background-color: white;'] * len(df.columns)  # Linhas ímpares - branco

# Estilizando a tabela
styled_table = (
    df.style
    .apply(highlight_header, axis=0)  # Aplicando estilo no cabeçalho
    .apply(lambda row: zebra_stripes(row.name), axis=1)  # Aplicando listras zebreadas nas linhas
    .set_properties(**{
        'border': '1px solid black',  # Grade preta nas células
        'padding': '8px',  # Espaçamento interno
        'text-align': 'center'  # Centralização do texto
    })
)

# Exibindo a tabela 
styled_table